In [57]:
from pathlib import Path
Path("HistoQuant/nuc_seg/raw/source").mkdir(parents=True, exist_ok=True)
Path("HistoQuant/nuc_seg/raw/mask").mkdir(parents=True, exist_ok=True)
Path("HistoQuant/nuc_seg/train/source").mkdir(parents=True, exist_ok=True)
Path("HistoQuant/nuc_seg/train/mask").mkdir(parents=True, exist_ok=True)
Path("HistoQuant/nuc_seg/test/source").mkdir(parents=True, exist_ok=True)
Path("HistoQuant/nuc_seg/test/mask").mkdir(parents=True, exist_ok=True)
Path("HistoQuant/nuc_seg/results").mkdir(parents=True, exist_ok=True)
Path("HistoQuant/nuc_seg/train_aug/source").mkdir(parents=True, exist_ok=True)
Path("HistoQuant/nuc_seg/train_aug/mask").mkdir(parents=True, exist_ok=True)

In [58]:
# Extract usefull file from archive
import os
import tarfile
img_archive = tarfile.open("EHRoes/image_data.tar.gz")
for file in img_archive.getmembers():
    if file.isreg():
        file.name = os.path.basename(file.name) 
        if ".gitkeep" in file.name: continue
        elif "images_db.csv" in file.name: continue
        elif "_blend_image.png" in file.name: continue
        elif "_classifier.pkl" in file.name: continue
        elif "_mask_annot.json" in file.name: continue
        elif "_seq_matrix.npy" in file.name: img_archive.extract(file, "HistoQuant/nuc_seg/raw/mask")
        #elif "_mask_image.png" in file.name: img_archive.extract(file, "HistoQuant/nuc_seg/raw/mask")
        elif "_mask_image.png" in file.name: pass
        else: img_archive.extract(file, "HistoQuant/nuc_seg/raw/source")

In [59]:
# Extract usefull file from archive
import os
import tarfile
img_archive = tarfile.open("HistoQuant/TNBC_NucleiSegmentation.tar.gz")
for file in img_archive.getmembers():
    if file.isreg():
        if "GT_" in file.name:
            file.name = os.path.basename(file.name) 
            img_archive.extract(file, "HistoQuant/nuc_seg/raw/mask")
        elif "Slide_" in file.name:
            file.name = os.path.basename(file.name) 
            img_archive.extract(file, "HistoQuant/nuc_seg/raw/source")

In [62]:
# Make Tuple of all (source img, mask img, numpy matrix)
import glob
all_raw = glob.glob("HistoQuant/nuc_seg/raw/source/*")
all_mask = glob.glob("HistoQuant/nuc_seg/raw/mask/*")
all_data_tuple = []
for i in range(len(all_mask)):
    mask_name = all_mask[i].split("/")[-1]
    if mask_name[-4:] == ".npy": img_name = mask_name[:-15]
    else: img_name = mask_name
    raw_path = "HistoQuant/nuc_seg/raw/source/"+img_name
    all_data_tuple.append((raw_path, all_mask[i]))

In [64]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(all_data_tuple, train_size=0.70, random_state=42)

In [84]:
import shutil
from PIL import Image, ImageOps
import numpy as np
import pandas as pd

def get_width_height(im, df, df_filename):
    width, height = im.size
    ratio_series = df[df["Number"] == df_filename]["Ratio_Scale"]
    if ratio_series.empty:
        return width, height
    resize_ratio = ratio_series.iloc[0]
    if resize_ratio == 1:
        return width, height
    width = round(width * resize_ratio)
    height = round(height * resize_ratio)
    return width, height

# Extract Images and convert all to same format, same name different folder
# Resize all images to same ratio/calibration
CABLIRATION_GOAL = 0.1612
df = pd.read_csv("HistoQuant/muscle_atlas_2_7_filt_triple.csv")
df["Ratio_Scale"] = df["Calibration (um/px)"] * 1/0.1612

for data_tuple in train:
    raw = data_tuple[0]
    mask = data_tuple[1]
    filename = raw.split("/")[-1] # Remove full path
    df_filename = ''.join(filename.split("_")[1:])
    filename = filename.split(".")[0] # Remove extension
    filename = filename + ".tif" # Add extension

    # Mask Processing
    if mask[-4:] == ".npy":
        npy_array = np.load(mask)
    else:
        mask_img = Image.open(mask)
        npy_array = np.array(mask_img)
    npy_array = np.where(npy_array == 177, 0, npy_array)
    npy_array = np.where(npy_array == 26, 255, npy_array)

    with Image.open(raw) as im:
        gray_image = ImageOps.grayscale(im)
        width, height = get_width_height(im, df, df_filename)
        gray_image = gray_image.resize((width, height), resample=Image.BICUBIC)
        gray_image.save("HistoQuant/nuc_seg/train/source/" + filename, "TIFF")
    with Image.fromarray(npy_array) as im:
        width, height = get_width_height(im, df, df_filename)
        im = im.resize((width, height), Image.NEAREST )
        im.save("HistoQuant/nuc_seg/train/mask/" + filename, "TIFF")

for data_tuple in test:
    raw = data_tuple[0]
    mask = data_tuple[1]
    filename = raw.split("/")[-1] # Remove full path
    df_filename = ''.join(filename.split("_")[1:])
    filename = filename.split(".")[0] # Remove extension
    filename = filename + ".tif" # Add extension

    # Mask Processing
    if mask[-4:] == ".npy":
        npy_array = np.load(mask)
    else:
        mask_img = Image.open(mask)
        npy_array = np.array(mask_img)
    npy_array = np.where(npy_array == 177, 0, npy_array)
    npy_array = np.where(npy_array == 26, 255, npy_array)

    with Image.open(raw) as im:
        gray_image = ImageOps.grayscale(im)
        width, height = get_width_height(im, df, df_filename)
        gray_image = gray_image.resize((width, height), resample=Image.BICUBIC)
        gray_image.save("HistoQuant/nuc_seg/test/source/" + filename, "TIFF")
    with Image.fromarray(npy_array) as im:
        width, height = get_width_height(im, df, df_filename)
        im = im.resize((width, height), resample=Image.NEAREST )
        im.save("HistoQuant/nuc_seg/test/mask/" + filename, "TIFF")